Simple word2vec model with 200 features for each utterance, not considering other features (speaker etc.)

In [21]:
import json
import gensim

path = r"C:\Users\user\git\NLPProj\Frames-dataset\chats"
sents = []
for filename in os.listdir(path):
    chat_file = open(os.path.join(path, filename), encoding='utf-8')
    chat = json.load(chat_file)
    if 'turns' in chat:
        for turn in chat['turns']:
            sents.append( turn['text'].split() )

model = gensim.models.Word2Vec(sents, workers=10, size=200, min_count=1)
model.vocab

{'amir': <gensim.models.word2vec.Vocab at 0x7ff1b1818fd0>,
 'iris': <gensim.models.word2vec.Vocab at 0x7ff1b1818c50>,
 'school': <gensim.models.word2vec.Vocab at 0x7ff1c3f80b70>,
 'to': <gensim.models.word2vec.Vocab at 0x7ff1b1818b38>,
 'went': <gensim.models.word2vec.Vocab at 0x7ff1c3f802b0>}

Define the RNN - TBD

In [ ]:
import keras

keras.layers.recurrent.SimpleRNN(128, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, 
                                 kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)